# Imports libs etc

In [209]:
import pandas as pd
import sys
sys.path.append('/Users/charilaostsarouchas/tsarouch_github/drivendata/pump_it_up')
%load_ext autoreload
%autoreload 2

from pyspark.sql import Row
from pyspark.mllib.regression import LabeledPoint


from config import data_path, train_values_file, train_labels_file, test_values_file
from reader import get_train_data_spark_df, get_test_data_spark_df
from ml_features import get_index_string_d, get_string_indexed_df
from ml_data_structures import get_supervised_learning_df
from dataframe_helper import change_dataframe_column_type

#from pyspark.sql.types import StringType, IntegerType, FloatType
#from pyspark.sql.functions import udf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read Data

In [210]:
df_train = get_train_data_spark_df(sqlContext)
df_test = get_test_data_spark_df(sqlContext)

# Prepare Data for ML

In [218]:
df_train_SL = get_supervised_learning_df(df_train, 'status_group', ['gps_height', 'latitude', 'longitude'])
df_train_SL = get_string_indexed_df(df_train_SL, ['label'])
label_index_string_d = get_index_string_d(df_train_SL, 'label')
# Run random forest
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(numTrees=3,
                            maxDepth=2, 
                            labelCol="label_index",
                            seed=42)
model = rf.fit(df_train_SL)

# Perform Random Forest on Test Data

In [224]:
df_test_SL = get_supervised_learning_df(df_test, 'id', ['gps_height', 'latitude', 'longitude'], label_name='id')
predictions_df = model.transform(df_test_SL)

In [225]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
predictions_df = predictions_df.withColumn(
            'status_group', 
            udf(lambda x : label_index_string_d[x], StringType())
            (predictions_df.prediction))

# Merge to the exact format to be uploaded

In [228]:
predictions_df = change_dataframe_column_type(predictions_df, 'id', 'int')

predictions_pdf = \
    predictions_df\
    .select(predictions_df['id'], predictions_df['status_group'])\
    .toPandas()
original_test_df = pd.read_csv(test_values_file, usecols=['id'])

# Merge to the required ids (sorted as in the test file) and dump to csv
pd.merge(original_test_df, predictions_pdf, on='id').to_csv(os.path.join(data_path, 'results.csv'), index=False)